<a href="https://colab.research.google.com/github/ranjiGT/Hitmonchan/blob/master/DNN_Wines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color='red'>***video link***</font> --> https://youtu.be/0lqjMFEtK78

#<font color='redr'> ***Buliding a Deep Neural Network on Wines Quality Dataset (Kaggle)</font>***

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("datasets_35901_52633_winequalityN.csv").dropna()
df

In [ ]:
df.dtypes

In [ ]:
df["type"].value_counts()

In [ ]:
red=df.loc[df["type"].isin(["red"])]
red

In [ ]:
white=df.loc[df["type"].isin(["white"])]
white

In [ ]:
pd.isnull(df).count()

In [ ]:
df["target"] = np.where(df["type"]=='red',1,0)
df

In [ ]:
import seaborn as sns
sns.pairplot(df, diag_kind='kde',hue='type',)

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split

x = df.iloc[:,1:13]

y = np.ravel(df.target) #return contiguous flattened array

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42) #random split into train and test set
print("Train_X: ",x_train.size)
print("Test_Y: ",y_test.size)
print("Test_X: ", x_test.size)
print("Train_Y: ", y_train.size)

In [ ]:
y_test[0:10]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler().fit(x_train) #std dev ; z-score; do not apply std scaling on output var
x_train=scaler.transform(x_train)
#print("Transformed X Train",x_train)
x_test=scaler.transform(x_test)
#print("Transformed X Test",x_test)

#<font color='red'> ***Neural Net-Building block***


> ***Blue print of the model***



In [ ]:
from tensorflow.keras import Sequential #2 ways Sequential=simpler, Functional=flexible; versatile
from tensorflow.keras.layers import Dense 
from keras.layers.advanced_activations import ReLU

#instatiation 
model=Sequential() # empty model ; Xavier methodology of intialization of weights

model.add(Dense(30, activation='relu', input_shape=(12,))) #1st hidden layer; 30 = no.of units/neurons ; no guideline for how many neurons to start with all HPT

#Newly added hidden layer
#model.add(Dense(20, activation='relu')) #add one at a time 

#output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

<font color = 'yellow'>***Model as a entire entity ready to build as a basis for performance evaluation***</font> 

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #values are hpt; exe object loss : error function; optimizer: ADAM HPTs; binary cross entropy is same as log-loss entropy
epochs=40 # 1 epoch = times you read dataset(train) not iterations; needs to do early stopping (mandat) also, many epochs leads to overfitting

#batch_size=50000
batch_size=50017
#batch_size=256

In [ ]:
model.summary() #names given internally by the complier; depends upon how many times you have run the code cell.

In [ ]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.3, verbose=True) #outlier analysis must be performed before execution of this line; perform clustering if by univariate analysis it's is not visible/detectable

loss,accuracy = model.evaluate(x_test, y_test, verbose=False) #invisible outliers=leverage points when you perform univariate analysis

#train and validation set

In [ ]:
print(history.history['accuracy'])
print(history.history['val_accuracy']) #history captues accuracy scores on train , test and loss

ta = pd.DataFrame(history.history['accuracy']) #training acc
va = pd.DataFrame(history.history['val_accuracy']) #validation acc

tva = pd.concat([ta,va], axis=1)
tva.boxplot()

In [ ]:
y_pred = np.round(model.predict(x_test))

In [ ]:
y_pred[:]

In [ ]:
loss, acc = model.evaluate(x_train,y_train,verbose=0)
print('Train accuracy: %.3f' %acc)

In [ ]:
loss, acc = model.evaluate(x_test,y_test,verbose=0) #verbose for showing the log
print('Test accuracy: %.3f' %acc) # %.3f for precision of the values

In [ ]:
from sklearn import metrics
print(pd.DataFrame(metrics.confusion_matrix(y_test, y_pred,labels=[0,1]), index=['true:White','true:Red'], columns=['pred:White', 'pred:Red']))